<a href="https://colab.research.google.com/github/Joyce-ZhouY/ECE1512-ProjectB/blob/main/ProjectB_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
!pip install pthflops
import torch
import torch.nn as nn
#from fvcore.nn import FlopCountAnalysis
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import time
import copy
from torchvision.utils import save_image
import os
from pthflops import count_ops
# import utility classes
import networks
import utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# initialize args
args = type('', (), {})()
args.batch_train = 256
args.batch_real = 256
args.epoch_eval_train = 30 # the number of epoch to train a model with synthetic dataset 
args.epoch_train = 10 # the number of epoch to train a network
args.lr_net = 0.01 # learning rate of the network
args.lr_img = 0.1 # learning rate of synthetic dataset
args.dsa_strategy = None
args.num_eval = 20 # number of random weight initialization
args.num_exp = 1 # number of experiments
args.ipc = 10 # image per class
args.outer_loop, args.inner_loop = utils.get_loops(args.ipc)
args.batch_real = 256 # batch size for original dataset
args.dsa = None
args.method = 'DC'
args.Iteration = 10 # training iterations
args.model = 'ConvNet'
args.dataset = 'MNIST'
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.dsa_param = utils.ParamDiffAug()
args.dsa = True if args.method == 'DSA' else False
args.dc_aug_param = None
args.dis_metric = 'ours' # distance metric
args.data_path = 'data'
args.save_path = 'result'

step a). train the selected model with original dataset

In [ ]:
args.dataset = 'CIFAR10'
args.model = 'ConvNet'
# Load dataset
# no augmentation
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, './data')

# load and train netwrok
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)


accurracy = []
it_eval = 1
_, train_acc, test_acc = utils.evaluate_dataset(it_eval, net, dst_train, testloader, args)
accurracy.append(test_acc)

def count_flops(dst_test):
  flops = 0
  images_test = []
  labels_test = []
  for i in range(len(dst_test)):
    images_test.append(torch.unsqueeze(dst_test[i][0], dim=0))
  for i in range(len(dst_test)):
    labels_test.append(dst_test[i][1])
  class_index = [[] for c in range(num_classes)]
  for n, label in enumerate(labels_test):
    class_index[label].append(n)
  images_test = torch.cat(images_test, dim=0).to(args.device)
  def pick_images(c, n):
      index = np.random.permutation(class_index[c])[:n]
      return images_test[index]
  for n in range(num_classes):
    img_batch = pick_images(n, 2560)
    flops += count_ops(net, img_batch)[0]
  return flops

flops = count_flops(dst_test)

print("Evaluate original dataset, accuracy = %.4f\n"%(np.mean(accurracy)))
print("The number of FLOPS = %.2f"%(flops))

In [6]:
args.dataset = 'MNIST'
args.model = 'ConvNet'
# Load dataset
# no augmentation
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, './data')

# load and train netwrok
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)

accurracy = []
it_eval = 1
_, train_acc, test_acc = utils.evaluate_dataset(it_eval, net, dst_train, testloader, args)
accurracy.append(test_acc)

def count_flops(dst_test):
  flops = 0
  images_test = []
  labels_test = []
  for i in range(len(dst_test)):
    images_test.append(torch.unsqueeze(dst_test[i][0], dim=0))
  for i in range(len(dst_test)):
    labels_test.append(dst_test[i][1])
  class_index = [[] for c in range(num_classes)]
  for n, label in enumerate(labels_test):
    class_index[label].append(n)
  images_test = torch.cat(images_test, dim=0).to(args.device)
  def pick_images(c, n):
      index = np.random.permutation(class_index[c])[:n]
      return images_test[index]
  for n in range(num_classes):
    img_batch = pick_images(n, 2560)
    flops += count_ops(net, img_batch)[0]
  return flops

flops = count_flops(dst_test)

print("Evaluate original dataset, accuracy = %.4f\n"%(np.mean(accurracy)))
print("The number of FLOPS = %.2f"%(flops))

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

[2022-12-12 01:49:19] Evaluate_01: epoch = 0010 train time = 132 s train loss = 0.014708 train acc = 0.9967, test acc = 0.9940
Operation     OPS           
------------  ------------  
features_0    1284505600    
features_2    256901120     
features_3    128450560     
features_4    37025873920   
features_6    64225280      
features_7    32112640      
features_8    9256468480    
features_10   16056320      
features_11   8028160       
classifier    20070410      
-----------   -----------   
Input size: (980, 1, 28, 28)
48,092,692,490 FLOPs or approx. 48.09 GFLOPs
Operation     OPS           
------------  ------------  
features_0    1487667200    
features_2    297533440     
features_3    148766720     
features_4    42882007040   
features_6    74383360      
features_7    37191680      
features_8    10720501760   
features_10   18595840      
features_11   9297920       
classifier    23244810      

step b). learn synthetic dataset

In [4]:
#Inspired by https://arxiv.org/abs/2006.05929
def dataset_compress():
  if not os.path.exists(args.save_path):
    os.mkdir(args.save_path)
  
  evaluate_pool = [args.Iteration]

  # load real dataset
  channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
  models = utils.get_eval_pool(args.eval_mode, args.model, args.model)

  # record accuracy of each model
  records = dict()

  for model in models:
    records[model] = []

  # start experiments loops
  for experiment in range(args.num_exp):
    # orgnize real data
    images_real = []
    labels_real = []
    for i in range(len(dst_train)):
      images_real.append(torch.unsqueeze(dst_train[i][0], dim=0))

    for i in range(len(dst_train)):
      labels_real.append(dst_train[i][1])

    class_index = [[] for c in range(num_classes)]
    for n, label in enumerate(labels_real):
      class_index[label].append(n)
    # move inputs to device
    images_real = torch.cat(images_real, dim=0).to(args.device)
    labels_real = torch.tensor(labels_real, dtype=torch.long, device=args.device)

    # implemente a method of ramdomly selecting n images from each class
    def pick_images(c, n):
      index = np.random.permutation(class_index[c])[:n]
      return images_real[index]

    # initialize synthetic dataset: gaussian noise
    images_syn = torch.randn(
        size = (args.ipc * num_classes, channel, im_size[0], im_size[1]),
        dtype = torch.float,
        requires_grad=True, 
        device=args.device
    )
    labels_syn = torch.tensor(
        [np.ones(args.ipc) * n for n in range(num_classes)], 
        dtype=torch.long, 
        requires_grad=False, 
        device=args.device
    ).view(-1)
  
    # if args.init = real, initilize synthetic data by ramdomly choosing from real data
    if args.init == 'real':
      for n in range(num_classes):
        images_syn.data[n * args.ipc : (n + 1) * args.ipc] = pick_images(n, args.ipc).detach().data


    # training process begins
    optimizer_syn = torch.optim.SGD(
      [images_syn, ], 
      lr=args.lr_img, 
      momentum=0.5) 
    optimizer_syn.zero_grad()
    loss_fn = nn.CrossEntropyLoss().to(args.device)

    for iter in range(args.Iteration + 1):
      # train synthetic data
      net = utils.get_network(args.model, channel, num_classes, im_size).to(args.device) # get a random model
      net.train()
      paras = list(net.parameters())
      optimizer_net = torch.optim.SGD(net.parameters(), lr=args.lr_net)  # optimizer_img for synthetic data
      optimizer_net.zero_grad()
      # loss_avg = 0

      # outer loop
      for outer in range(args.outer_loop):
        loss = torch.tensor(0.0).to(args.device)
        for n in range(num_classes):
          # get a pair of batch real and synthetic 
          image_batch_real = pick_images(n, args.batch_real)
          label_batch_real = torch.ones((image_batch_real.shape[0],), device=args.device, dtype=torch.long) * n
          image_batch_syn = images_syn[n * args.ipc : (n + 1) * args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))
          label_batch_syn = torch.ones((args.ipc,), device=args.device, dtype=torch.long) * n

          out_real = net(image_batch_real)
          loss_real = loss_fn(out_real, label_batch_real)
          gradients_real = torch.autograd.grad(loss_real, paras)
          gradients_real = list((_.detach().clone() for _ in gradients_real))

          out_syn = net(image_batch_syn)
          loss_syn = loss_fn(out_syn, label_batch_syn)
          gradients_syn = torch.autograd.grad(loss_syn, paras, create_graph=True)

          loss += utils.match_loss(gradients_syn, gradients_real, args)

        optimizer_syn.zero_grad()
        loss.backward()
        optimizer_net.step()

        # update model
        images_train, labels_train = copy.deepcopy(images_syn.detach()), copy.deepcopy(labels_syn.detach())
        dst_train_syn = utils.TensorDataset(images_train, labels_train)
        trainloader = torch.utils.data.DataLoader(dst_train_syn, batch_size=args.batch_train, shuffle=True, num_workers=0)

        # inner loop
        for inner in range(args.inner_loop):
          utils.epoch('train', trainloader, net, optimizer_net, loss_fn, args, aug = True if args.dsa else False)
            
      # evaluate synthetic set 
      if iter in evaluate_pool:
        for model in models:
          args.epoch_eval_train = 300
        
          accuracy = []
          # loop over number of random model initialization
          for eva in range(args.num_eval):
            # load a network
            net = utils.get_network(model, channel, num_classes, im_size).to(args.device)
            _, acc_train, acc_test = utils.evaluate_synset(eva, net, images_syn, labels_syn, testloader, args)
            accuracy.append(acc_test)
          print('Evaluate synthetic data on model: %s, mean accuracy = %.4f'%(model, np.mean(accuracy)))

          # save the last accuracy into records
          if iter == args.Iteration:
            records[model] += accuracy

        #save synthetic data
        path =  os.path.join(args.save_path, 'gradient_syn_%s_%s_%s_%dipc_exp%d_iter%d.png'%('random' if args.init == 'real' else 'noise', args.dataset, args.model, args.ipc, experiment, iter))
        save_image(images_syn, path, nrow=args.ipc)


  for model in models:
    accuracy = records[model]
    print("Experiments = %d, model = %s, accuracy= %.2f"%(args.num_exp, args.model, np.mean(accuracy)*100))
  return images_syn, labels_syn

step c). initialize the synthetic dataset by randomly selecting from real data

In [6]:
args.model = 'ConvNet'
args.dataset = 'MNIST'
args.eval_mode = 'S'
args.init = 'real'
syn_MNIST_real = dataset_compress()

<ipython-input-4-4970ec5c2553>:48: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  labels_syn = torch.tensor(


[2022-12-12 19:31:04] Evaluate_00: epoch = 0300 train time = 3 s train loss = 0.003882 train acc = 1.0000, test acc = 0.9040
[2022-12-12 19:31:10] Evaluate_01: epoch = 0300 train time = 3 s train loss = 0.003895 train acc = 1.0000, test acc = 0.8989
[2022-12-12 19:31:16] Evaluate_02: epoch = 0300 train time = 3 s train loss = 0.003773 train acc = 1.0000, test acc = 0.8975
[2022-12-12 19:31:21] Evaluate_03: epoch = 0300 train time = 3 s train loss = 0.003906 train acc = 1.0000, test acc = 0.8960
[2022-12-12 19:31:27] Evaluate_04: epoch = 0300 train time = 3 s train loss = 0.003949 train acc = 1.0000, test acc = 0.9018
[2022-12-12 19:31:33] Evaluate_05: epoch = 0300 train time = 3 s train loss = 0.003815 train acc = 1.0000, test acc = 0.9003
[2022-12-12 19:31:39] Evaluate_06: epoch = 0300 train time = 3 s train loss = 0.003890 train acc = 1.0000, test acc = 0.8984
[2022-12-12 19:31:45] Evaluate_07: epoch = 0300 train time = 3 s train loss = 0.004322 train acc = 1.0000, test acc = 0.8934


In [38]:
args.model = 'ConvNet'
args.dataset = 'CIFAR10'
args.eval_mode = 'S'
args.init = 'real'
syn_CIFAR10_real = dataset_compress()

Files already downloaded and verified
Files already downloaded and verified


<ipython-input-37-27a923bf4953>:52: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  labels_syn = torch.tensor(


[2022-12-12 17:40:38] Evaluate_00: epoch = 0300 train time = 3 s train loss = 0.004761 train acc = 1.0000, test acc = 0.2433
[2022-12-12 17:40:44] Evaluate_01: epoch = 0300 train time = 3 s train loss = 0.004910 train acc = 1.0000, test acc = 0.2435
[2022-12-12 17:40:50] Evaluate_02: epoch = 0300 train time = 3 s train loss = 0.004976 train acc = 1.0000, test acc = 0.2418
[2022-12-12 17:40:56] Evaluate_03: epoch = 0300 train time = 3 s train loss = 0.005119 train acc = 1.0000, test acc = 0.2396
[2022-12-12 17:41:03] Evaluate_04: epoch = 0300 train time = 3 s train loss = 0.004947 train acc = 1.0000, test acc = 0.2419
[2022-12-12 17:41:09] Evaluate_05: epoch = 0300 train time = 3 s train loss = 0.005122 train acc = 1.0000, test acc = 0.2356
[2022-12-12 17:41:15] Evaluate_06: epoch = 0300 train time = 3 s train loss = 0.005050 train acc = 1.0000, test acc = 0.2441
[2022-12-12 17:41:21] Evaluate_07: epoch = 0300 train time = 3 s train loss = 0.005071 train acc = 1.0000, test acc = 0.2420


step d). initialize the synthetic dataset with Gaussian Noise

In [39]:
args.dataset = 'MNIST'
args.eval_mode = 'S'
args.init = 'noise'
syn_MNIST_noise = dataset_compress()

<ipython-input-37-27a923bf4953>:52: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  labels_syn = torch.tensor(


[2022-12-12 17:53:25] Evaluate_00: epoch = 0300 train time = 3 s train loss = 0.004635 train acc = 1.0000, test acc = 0.2193
[2022-12-12 17:53:31] Evaluate_01: epoch = 0300 train time = 3 s train loss = 0.004631 train acc = 1.0000, test acc = 0.1504
[2022-12-12 17:53:37] Evaluate_02: epoch = 0300 train time = 3 s train loss = 0.004604 train acc = 1.0000, test acc = 0.1117
[2022-12-12 17:53:43] Evaluate_03: epoch = 0300 train time = 3 s train loss = 0.004604 train acc = 1.0000, test acc = 0.1664
[2022-12-12 17:53:49] Evaluate_04: epoch = 0300 train time = 3 s train loss = 0.004565 train acc = 1.0000, test acc = 0.1027
[2022-12-12 17:53:55] Evaluate_05: epoch = 0300 train time = 3 s train loss = 0.004730 train acc = 1.0000, test acc = 0.1439
[2022-12-12 17:54:01] Evaluate_06: epoch = 0300 train time = 3 s train loss = 0.004500 train acc = 1.0000, test acc = 0.0858
[2022-12-12 17:54:07] Evaluate_07: epoch = 0300 train time = 3 s train loss = 0.004633 train acc = 1.0000, test acc = 0.1490


In [40]:
args.dataset = 'CIFAR10'
args.eval_mode = 'S'
args.init = 'noise'
syn_CIFAR10_noise = dataset_compress()

Files already downloaded and verified
Files already downloaded and verified


<ipython-input-37-27a923bf4953>:52: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  labels_syn = torch.tensor(


[2022-12-12 18:06:49] Evaluate_00: epoch = 0300 train time = 3 s train loss = 0.003793 train acc = 1.0000, test acc = 0.0754
[2022-12-12 18:06:55] Evaluate_01: epoch = 0300 train time = 3 s train loss = 0.003797 train acc = 1.0000, test acc = 0.0882
[2022-12-12 18:07:02] Evaluate_02: epoch = 0300 train time = 3 s train loss = 0.003732 train acc = 1.0000, test acc = 0.0868
[2022-12-12 18:07:08] Evaluate_03: epoch = 0300 train time = 3 s train loss = 0.003785 train acc = 1.0000, test acc = 0.0849
[2022-12-12 18:07:14] Evaluate_04: epoch = 0300 train time = 3 s train loss = 0.003871 train acc = 1.0000, test acc = 0.0946
[2022-12-12 18:07:21] Evaluate_05: epoch = 0300 train time = 3 s train loss = 0.003810 train acc = 1.0000, test acc = 0.0858
[2022-12-12 18:07:27] Evaluate_06: epoch = 0300 train time = 3 s train loss = 0.003743 train acc = 1.0000, test acc = 0.1015
[2022-12-12 18:07:33] Evaluate_07: epoch = 0300 train time = 3 s train loss = 0.003754 train acc = 1.0000, test acc = 0.0811


step e): train the selected network from scratch on the condensed images and compare the results

In [41]:
# train a network from scratch with synthetic dataset
# evaluate on real dataset
# testing data
args.dataset = 'MNIST'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
it_eval = args.num_eval
args.model = 'ConvNet'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)

images_train = syn_MNIST_real[0].to(args.device)
labels_train = syn_MNIST_real[1].to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)

_, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
print("test with real dataset, accuracy = %.4f"%(acc_test))

[2022-12-12 18:18:16] Evaluate_100: epoch = 0010 train time = 0 s train loss = 1.104398 train acc = 0.8900, test acc = 0.7584
test with real dataset, accuracy = 0.7584


In [42]:
# train a network from scratch with synthetic dataset
# evaluate on real dataset
# testing data
args.dataset = 'CIFAR10'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)

it_eval = args.num_eval
args.model = 'ConvNet'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)

images_train = syn_CIFAR10_real[0].to(args.device)
labels_train = syn_CIFAR10_real[1].to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)

_, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
print("test with real dataset, accuracy = %.4f"%(acc_test))

Files already downloaded and verified
Files already downloaded and verified
[2022-12-12 18:20:25] Evaluate_100: epoch = 0010 train time = 0 s train loss = 1.257839 train acc = 0.8300, test acc = 0.1969
test with real dataset, accuracy = 0.1969


**Cross-architecture Generalization**

In [8]:
# choose another network and train it with learned synthetic dataset 
# testing data
args.dataset = 'MNIST'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'MLP'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = syn_MNIST_real[0].to(args.device)
labels_train = syn_MNIST_real[1].to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)

_, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))


[2022-12-12 19:42:15] Evaluate_20: epoch = 0010 train time = 0 s train loss = 1.111644 train acc = 0.7500, test acc = 0.6382
train with synthetic data, accuracy = 0.6382


In [44]:
# choose another network and train it with learned synthetic dataset 
# testing data
args.dataset = 'MNIST'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'AlexNet'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = syn_MNIST_real[0].to(args.device)
labels_train = syn_MNIST_real[1].to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)

_, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))


[2022-12-12 18:21:31] Evaluate_100: epoch = 0010 train time = 0 s train loss = 2.252966 train acc = 0.1000, test acc = 0.0958
train with synthetic data, accuracy = 0.0958


In [45]:
# choose another network and train it with learned synthetic dataset 
# testing data
args.dataset = 'MNIST'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'VGG11'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = syn_MNIST_real[0].to(args.device)
labels_train = syn_MNIST_real[1].to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)

_, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))

[2022-12-12 18:21:41] Evaluate_100: epoch = 0010 train time = 0 s train loss = 2.833203 train acc = 0.1100, test acc = 0.1135
train with synthetic data, accuracy = 0.1135


In [46]:
# choose another network and train it with learned synthetic dataset 
# testing data
args.dataset = 'MNIST'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'ResNet18'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = syn_MNIST_real[0].to(args.device)
labels_train = syn_MNIST_real[1].to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)

_, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))

[2022-12-12 18:22:05] Evaluate_100: epoch = 0010 train time = 1 s train loss = 1.206146 train acc = 0.9100, test acc = 0.5305
train with synthetic data, accuracy = 0.5305


In [47]:
# choose another network and train it with learned synthetic dataset 
# testing data
args.dataset = 'MNIST'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'ConvNetW256'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = syn_MNIST_real[0].to(args.device)
labels_train = syn_MNIST_real[1].to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)

_, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))

[2022-12-12 18:22:15] Evaluate_100: epoch = 0010 train time = 0 s train loss = 0.212914 train acc = 0.9200, test acc = 0.7513
train with synthetic data, accuracy = 0.7513


In [48]:
# choose another network and train it with learned synthetic dataset 
# testing data
args.dataset = 'CIFAR10'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'MLP'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = syn_CIFAR10_real[0].to(args.device)
labels_train = syn_CIFAR10_real[1].to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)

_, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))


Files already downloaded and verified
Files already downloaded and verified
[2022-12-12 18:22:53] Evaluate_100: epoch = 0010 train time = 0 s train loss = 0.781122 train acc = 0.8900, test acc = 0.2253
train with synthetic data, accuracy = 0.2253


In [49]:
# choose another network and train it with learned synthetic dataset 
# testing data
args.dataset = 'CIFAR10'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'AlexNet'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = syn_CIFAR10_real[0].to(args.device)
labels_train = syn_CIFAR10_real[1].to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)

_, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))


Files already downloaded and verified
Files already downloaded and verified
[2022-12-12 18:24:16] Evaluate_100: epoch = 0010 train time = 0 s train loss = 2.239855 train acc = 0.1300, test acc = 0.1031
train with synthetic data, accuracy = 0.1031


In [51]:
# choose another network and train it with learned synthetic dataset 
# testing data
args.dataset = 'CIFAR10'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'VGG11'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = syn_CIFAR10_real[0].to(args.device)
labels_train = syn_CIFAR10_real[1].to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)

_, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))


Files already downloaded and verified
Files already downloaded and verified
[2022-12-12 18:25:17] Evaluate_100: epoch = 0010 train time = 0 s train loss = 2.921377 train acc = 0.1000, test acc = 0.1000
train with synthetic data, accuracy = 0.1000


In [52]:
# choose another network and train it with learned synthetic dataset 
# testing data
args.dataset = 'CIFAR10'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'ResNet18'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = syn_CIFAR10_real[0].to(args.device)
labels_train = syn_CIFAR10_real[1].to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)

_, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))


Files already downloaded and verified
Files already downloaded and verified
[2022-12-12 18:26:02] Evaluate_100: epoch = 0010 train time = 1 s train loss = 1.335190 train acc = 0.9600, test acc = 0.1746
train with synthetic data, accuracy = 0.1746


In [54]:
# choose another network and train it with learned synthetic dataset 
# testing data
args.dataset = 'CIFAR10'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.batch_train = 32

args.model = 'ConvNetW256'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval

images_train = syn_CIFAR10_real[0].to(args.device)
labels_train = syn_CIFAR10_real[1].to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)

_, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
print("train with synthetic data, accuracy = %.4f"%(acc_test))


Files already downloaded and verified
Files already downloaded and verified
[2022-12-12 18:27:26] Evaluate_100: epoch = 0010 train time = 0 s train loss = 0.113526 train acc = 0.9800, test acc = 0.1978
train with synthetic data, accuracy = 0.1978


# Train Light Weight Model using synthetic data

In [35]:
#Inspired by https://arxiv.org/pdf/2209.14609.pdf

from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from torchvision import models
from torchsummary import summary


# train a network from scratch with synthetic dataset
# evaluate on real dataset
args.dataset = 'MNIST'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
args.model = 'MLP'
net = utils.get_network(args.model,channel=channel,num_classes=num_classes,im_size=im_size).to(args.device)
it_eval = args.num_eval
images_train = syn_MNIST_real[0].to(args.device)
labels_train = syn_MNIST_real[1].to(args.device)
dst_train = utils.TensorDataset(images_train, labels_train)
summary(net, (28, 28))

_, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
print("test with real dataset, accuracy = %.4f"%(acc_test))



# pruning network
parameters_to_prune = (
    (net.fc_1, 'weight'),
    (net.fc_2, 'weight'),
    (net.fc_3, 'weight'),
)
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.82,
)
print(    "Sparsity in fc1.weight: {:.2f}%".format(        100. * float(torch.sum(net.fc_1.weight == 0))        / float(net.fc_1.weight.nelement())    ))
print(    "Sparsity in fc2.weight: {:.2f}%".format(        100. * float(torch.sum(net.fc_2.weight == 0))        / float(net.fc_2.weight.nelement())    ))
print(    "Sparsity in fc3.weight: {:.2f}%".format(        100. * float(torch.sum(net.fc_3.weight == 0))        / float(net.fc_3.weight.nelement())    ))


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(args.device)
            y = y.to(args.device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct

# testing data
# args.dataset = 'MNIST'
# channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(args.dataset, args.data_path)
# _, acc_train, acc_test = utils.evaluate_dataset(it_eval,net,dst_train,testloader,args)
loss_fn = nn.CrossEntropyLoss().to(args.device)
acc_test = test_loop(testloader, net, loss_fn)
print("test with real dataset, accuracy = %.4f"%(acc_test))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]         100,480
            Linear-2                  [-1, 128]          16,512
            Linear-3                   [-1, 10]           1,290
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.45
Estimated Total Size (MB): 0.46
----------------------------------------------------------------
[2022-12-12 19:59:20] Evaluate_20: epoch = 0010 train time = 0 s train loss = 1.243565 train acc = 0.7600, test acc = 0.6350
test with real dataset, accuracy = 0.6350
Sparsity in fc1.weight: 90.02%
Sparsity in fc2.weight: 36.61%
Sparsity in fc3.weight: 34.22%
Test Error: 
 Accuracy: 66.0%, Avg loss: 1.694113 

test with real dataset, accuracy = 0.6602
